# Importer les librairies, et fichiers .py auxiliaires

In [ ]:
import numpy as np
from preprocess_data import *
from bayes_classifier import BayesClassifier
from complement_naive_bayes import train_cnb_with_tfidf, train_cnb
from ensemble_learning import train_ensemble
from xgboost_classifier import train_xgboost, train_xgboost_with_tfidf

from save_output import save_output

# Exploration de données

# Jalon 1) Naive de Bayes vanille
K-Fold Validation croisée ($k = 7)$:
- α: Lissage du postérieur de Bayes

In [ ]:
data_preprocess = DataPreprocess()
bayes_classifier = BayesClassifier()
best_lps, best_f1 = bayes_classifier.hyperparameter_tuning(data_preprocess.train, data_preprocess.label_train)
print(best_lps, best_f1)
tuned_bayes_classifier = BayesClassifier()
tuned_bayes_classifier.fit(data_preprocess.train, data_preprocess.label_train, best_lps)
predictions = np.array([tuned_bayes_classifier.predict(x_i) for x_i in data_preprocess.test])
save_output(predictions, "bayes", best_lps, "stopwords")

# Jalon 2)

Naive Bayes Complement (avec TFIDF)

In [ ]:
data_preprocess = DataPreprocess()
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(data_preprocess.train, data_preprocess.label_train)
tfidf_test = tfidf_transformer.transform(data_preprocess.test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_50_iter", "tfidf")

Naive Bayes Complement

In [ ]:
data_preprocess = DataPreprocess()
complement_naive_bayes = train_cnb(data_preprocess.train, data_preprocess.label_train)
predictions = complement_naive_bayes.predict(data_preprocess.test)
save_output(predictions, "cnb", "random_search_50_iter", "no_preprocessing")

XGBoost - Sur-échantillonnage bootstrap

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
x_train_oversampled, y_train_oversampled = boostrap_oversampling(data_preprocess.train, data_preprocess.label_train)
xgboost_classifier, tfidf_transformer = train_xgboost(x_train_oversampled, y_train_oversampled)
predictions = xgboost_classifier.predict(data_preprocess.test)
save_output(predictions, "xgboost", "random_search_15_iter", "stopwords_bootstrap")

# Apprentissage par Ensemble

CNB, XGBoost, Logistic Regression sans stopwords, sous-échantillonage

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords")